## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Lincoln,AU,-34.7333,135.8667,64.74,74,100,12.19,moderate rain
1,1,Zlobin,HR,45.2924,14.6541,42.10,67,20,20.71,few clouds
2,2,Tura,IN,25.5198,90.2201,67.50,66,0,1.83,clear sky
3,3,Byron Bay,AU,-28.6500,153.6167,77.70,75,33,9.35,scattered clouds
4,4,Punta Arenas,CL,-53.1500,-70.9167,42.91,56,75,4.61,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Byron Bay,AU,-28.6500,153.6167,77.70,75,33,9.35,scattered clouds
7,7,Georgetown,MY,5.4112,100.3354,82.33,90,20,1.01,few clouds
14,14,Atuona,PF,-9.8000,-139.0333,77.29,77,100,8.66,light rain
23,23,El Pinon,CO,10.4028,-74.8242,80.74,91,100,5.12,overcast clouds
26,26,Faanui,PF,-16.4833,-151.7500,81.28,73,100,7.18,light rain
30,30,Noumea,NC,-22.2763,166.4572,89.58,78,100,11.10,light rain
33,33,Avarua,CK,-21.2078,-159.7750,78.85,69,90,9.22,overcast clouds
40,40,Tanout,NE,14.9709,8.8879,76.01,19,0,9.42,clear sky
41,41,Kapaa,US,22.0752,-159.3190,80.58,75,10,1.99,clear sky
42,42,Ixtapa,MX,20.7000,-105.2000,77.23,81,28,2.51,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                156
City                   156
Country                156
Lat                    156
Lng                    156
Max Temp               156
Humidity               156
Cloudiness             156
Wind Speed             156
Current Description    156
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""

In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("", float("NaN"), inplace = True)
clean_hotel_df = hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Byron Bay,AU,77.70,scattered clouds,-28.6500,153.6167,Backpackers Inn on the Beach
7,Georgetown,MY,82.33,few clouds,5.4112,100.3354,Cititel Penang
14,Atuona,PF,77.29,light rain,-9.8000,-139.0333,Villa Enata
23,El Pinon,CO,80.74,overcast clouds,10.4028,-74.8242,Hotel Sol
41,Kapaa,US,80.58,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
45,Itarema,BR,80.33,overcast clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
51,Sumbe,AO,76.42,overcast clouds,-11.2061,13.8437,Hotel Ritz Sumbe
70,Kavieng,PG,84.40,light rain,-2.5744,150.7967,Nusa Island Retreat
80,Hithadhoo,MV,81.41,moderate rain,-0.6000,73.0833,Scoop Guest House
84,Santa Marta,CO,79.16,overcast clouds,11.2408,-74.1990,Casa de Isabella


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))